In [2]:
from datasets import load_dataset
import torch
from utils.translation_transformer import TransformerConfig

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {DEVICE}")

ds = load_dataset("wmt/wmt14", "de-en")

vocab_size = 30_000
vocab_path = "./data/bpe_tokenizer.json"

training_samples = len(ds["train"])
batch_size = 64
dataset_max_sample_len = 100

sharedVocab = True
# bpe_v3_ep12
configSmall = TransformerConfig(
 d_model=256,
 nhead=8,
 num_encoder_layers=4,
 num_decoder_layers=4,
 dim_feedforward=1024,
 dropout=0.1,
 max_len=150
)
# base model according to the paper 'Attention is all you need'
# big_3.8770loss
configBig = TransformerConfig(
    d_model=512,
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_feedforward=2048,
    dropout=0.1,
    max_len=150
)

# training
num_steps = 20_000
warmup_steps = 2_000
eval_iters = 10
patience = 1_000

label_smoothing = 0.1

# optimizer
start_lr = 3e-4
betas = (0.9, 0.98)
epsilon = 1e-9

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

de-en/train-00000-of-00003.parquet:   0%|          | 0.00/280M [00:00<?, ?B/s]

de-en/train-00001-of-00003.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

...

de-en/validation-00000-of-00001.parquet:   0%|          | 0.00/474k [00:00<?, ?B/s]

de-en/test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [3]:
from tokenizers import Tokenizer as HFTokenizer, decoders
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Metaspace
from utils.tokenization_vocab import HFTokenizerWrapper, Tokenizer
from pathlib import Path

bpe_tokenizer = HFTokenizer(BPE(unk_token=Tokenizer.UNK_TOKEN))
trainer = BpeTrainer(
    special_tokens=[Tokenizer.PAD_TOKEN, Tokenizer.SOS_TOKEN, Tokenizer.EOS_TOKEN, Tokenizer.UNK_TOKEN],
    vocab_size=vocab_size,
    show_progress=True
)

bpe_tokenizer.pre_tokenizer = Metaspace()
bpe_tokenizer.decoder = decoders.Metaspace()

pretrained = True  # Set to True if you want to load a previously saved tokenizer

Path(vocab_path).parent.mkdir(parents=True, exist_ok=True)

if Path(vocab_path).is_file():
    pretrained = True

if pretrained:
    bpe_tokenizer = HFTokenizer.from_file(vocab_path)
else:
    bpe_tokenizer.train(
        [
            './datasets/wmt14_translate_de-en_test.csv',
            './datasets/wmt14_translate_de-en_train.csv',
            './datasets/wmt14_translate_de-en_validation.csv',
        ],
        trainer=trainer
    )

    bpe_tokenizer.save(vocab_path)


tokenizer = HFTokenizerWrapper(bpe_tokenizer)

print(f"Vocab size: {bpe_tokenizer.get_vocab_size()}")

Vocab size: 30000


In [4]:
from utils.parallel_corpus import TranslationDataset, DataLoaderFactory, LazyTranslationPairs
from utils.tokenization_vocab import HFTokenizerWrapper
import os

# Create lazy wrappers - no materialization into lists!
train_src = LazyTranslationPairs(ds['train'], src_lang='de', tgt_lang='en', mode='src')
train_tgt = LazyTranslationPairs(ds['train'], src_lang='de', tgt_lang='en', mode='tgt')

test_src = LazyTranslationPairs(ds['test'], src_lang='de', tgt_lang='en', mode='src')
test_tgt = LazyTranslationPairs(ds['test'], src_lang='de', tgt_lang='en', mode='tgt')

tokenizer = HFTokenizerWrapper(bpe_tokenizer)

# Create datasets with lazy loading (processes on-the-fly, no upfront preprocessing)
train_ds = TranslationDataset(
    source_sentences=train_src,
    target_sentences=train_tgt,
    source_tokenizer=tokenizer,
    target_tokenizer=tokenizer,
    max_length=dataset_max_sample_len,
    lazy=True  # Enable lazy loading!
)

test_ds = TranslationDataset(
    source_sentences=test_src,
    target_sentences=test_tgt,
    source_tokenizer=tokenizer,
    target_tokenizer=tokenizer,
    max_length=dataset_max_sample_len,
    lazy=True
)

# Optimize num_workers based on CPU cores
optimal_workers = min(8, os.cpu_count() or 4)

train_loader = DataLoaderFactory.create_dataloader(
    dataset=train_ds,
    batch_size=batch_size,
    pad_idx=tokenizer.pad_idx,
    num_workers=optimal_workers,
    shuffle=True,  # Shuffle for training
    persistent_workers=True,  # Keep workers alive between epochs
    prefetch_factor=4  # Prefetch more batches
)

test_loader = DataLoaderFactory.create_dataloader(
    dataset=test_ds,
    batch_size=batch_size,
    pad_idx=tokenizer.pad_idx,
    num_workers=optimal_workers,
    shuffle=False,  # No shuffle for testing
    persistent_workers=True,
    prefetch_factor=4
)

print(f"✓ Lazy loading enabled - no memory materialization!")
print(f"✓ Using {optimal_workers} workers for parallel processing")
print(f"Train samples: {len(train_ds):,}, Test samples: {len(test_ds):,}")
print(f"Train batches: {len(train_loader):,}, Test batches: {len(test_loader):,}")

Initialized lazy dataset with 4508785 sentence pairs
Initialized lazy dataset with 3003 sentence pairs
✓ Lazy loading enabled - no memory materialization!
✓ Using 2 workers for parallel processing
Train samples: 4,508,785, Test samples: 3,003
Train batches: 70,450, Test batches: 47


In [5]:
# Import the TranslationTransformer
from utils.translation_transformer import TranslationTransformer, TranslationTransformerPytorch

# Initialize the model with larger max_len to handle max_length + special tokens
model = TranslationTransformer(
    src_vocab_size=len(tokenizer),
    tgt_vocab_size=len(tokenizer),
    config=configBig,
    padding_idx=tokenizer.pad_idx,
    sharedVocab=sharedVocab
)

print(f"Model initialized!")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

15,360,000 parameters in shared embedding
18,895,872 parameters in encoder layers
25,190,400 parameters in decoder layers
512 parameters in encoder norm layer
512 parameters in decoder norm layer
15,360,000 parameters in output projection
Model initialized!
Total parameters: 59,447,296


### Load model from checkpoint

In [ ]:
state_dict = torch.load("./models/best_model.pt", map_location=DEVICE)['model_state_dict']
new_state_dict = {
    k.replace("_orig_mod.", ""): v
    for k, v in state_dict.items()
}
model.load_state_dict(new_state_dict)
model.eval()

In [6]:
# Apply PyTorch optimizations
import torch

# 1. Enable TF32 for faster matmul on Ampere+ GPUs (A100, RTX 3090, etc.)
# This provides ~2x speedup for matrix multiplications with minimal accuracy loss
# torch.set_float32_matmul_precision('high')  # Options: 'highest', 'high', 'medium'
# torch.backends.fp32_precision = 'tf32'

# 2. For MPS (Apple Silicon), ensure we're using optimal settings
if DEVICE.type == "mps":
    # MPS backend is already optimized, but we can ensure memory efficiency
    torch.mps.empty_cache()  # Clear any cached memory
elif DEVICE.type == "cuda":
    # Enable TF32 for cuDNN convolutions as well
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
else:
    print("✓ Running on CPU (no GPU optimizations)")

model_compiled = torch.compile(model, mode='default')  # Options: 'default', 'reduce-overhead', 'max-autotune'

# Move model to device (GPU if available)
model = model.to(DEVICE)
model.train()

print(f"Using device: {DEVICE}")
print(f"Model moved to {DEVICE}")

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


Using device: cuda
Model moved to cuda


In [ ]:
import torch.nn as nn
import torch.optim as optim
from utils.train import train
from torch.optim.lr_scheduler import LambdaLR


def lr_lambda(step, warmup_steps=4000):
    """Learning rate schedule with warmup and decay."""
    step = max(step, 1)
    return configBig.d_model**(-0.5) * min(
        step ** -0.5,
        step * warmup_steps ** -1.5
    )

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_idx, label_smoothing=label_smoothing)
optimizer = optim.Adam(model.parameters(), lr=1, betas=betas, eps=epsilon)

scheduler = LambdaLR(optimizer, lambda step: lr_lambda(step, warmup_steps))

# Training
train_losses, best_loss = train(
    model=model_compiled,
    config=configBig,
    train_loader=train_loader,
    test_loader=test_loader,
    dataset_size=len(train_ds),
    criterion=criterion, 
    optimizer=optimizer,
    scheduler=scheduler,
    device=DEVICE,
    num_steps=num_steps,
    eval_iters=eval_iters,
    patience=patience
)

Starting training for 20,000 steps...
Total batches per epoch: 70,450
Dataset size: 4,508,785 samples
Learning rate: 0.000000 (with warmup and decay)


W0114 15:45:01.648000 1045 torch/_inductor/utils.py:1558] [0/0] Not enough SMs to use max_autotune_gemm mode
W0114 15:45:24.871000 1045 torch/fx/experimental/symbolic_shapes.py:6833] [0/1] _maybe_guard_rel() was called on non-relation expression Eq(s25, 1) | Eq(s98, s25)
W0114 15:47:21.763000 1045 torch/fx/experimental/symbolic_shapes.py:6833] [0/2] _maybe_guard_rel() was called on non-relation expression Eq(s25, 1) | Eq(s98, s25)


[Step 100/70450] - Training Loss: 9.7653, Validation Loss: 9.0999, Time/step: 1.56sec, lr: 0.00004941
[Step 200/70450] - Training Loss: 8.2757, Validation Loss: 7.9862, Time/step: 0.66sec, lr: 0.00009882
[Step 300/70450] - Training Loss: 7.6564, Validation Loss: 7.7567, Time/step: 0.67sec, lr: 0.00014823
[Step 400/70450] - Training Loss: 7.3834, Validation Loss: 7.5326, Time/step: 0.66sec, lr: 0.00019764
[Step 500/70450] - Training Loss: 7.1402, Validation Loss: 7.3492, Time/step: 0.66sec, lr: 0.00024705
[Step 600/70450] - Training Loss: 6.9601, Validation Loss: 7.2296, Time/step: 0.66sec, lr: 0.00029646
[Step 700/70450] - Training Loss: 6.8545, Validation Loss: 7.1480, Time/step: 0.67sec, lr: 0.00034587
[Step 800/70450] - Training Loss: 6.7196, Validation Loss: 7.0571, Time/step: 0.66sec, lr: 0.00039528
[Step 900/70450] - Training Loss: 6.6505, Validation Loss: 7.0022, Time/step: 0.66sec, lr: 0.00044470
[Step 1000/70450] - Training Loss: 6.5662, Validation Loss: 6.9007, Time/step: 0.6

In [ ]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
        'name': name,
        'mimeType': 'application/octet-stream'
    }

    media = MediaFileUpload(
        path, 
        mimetype='application/octet-stream',
        resumable=True
    )

    created = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()

    print('File ID: {}'.format(created.get('id')))
    return created

save_file_to_drive('best_model.zip', 'models/best_model.zip')

In [ ]:
# Plot training progress
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, marker='o', linewidth=2, markersize=6)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Training Loss', fontsize=12)
plt.title('Training Loss Over Time', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTraining Summary:")
print(f"  Initial Loss: {train_losses[0]:.4f}")
print(f"  Final Loss: {train_losses[-1]:.4f}")
print(f"  Best Loss: {best_loss:.4f}")
print(f"  Improvement: {((train_losses[0] - train_losses[-1]) / train_losses[0] * 100):.1f}%")

In [ ]:
# Test translation on sample input
import torch
import sacrebleu

@torch.no_grad()
def translate_sample(sentence: str, model: TranslationTransformer,
                     src_tokenizer: HFTokenizerWrapper, tgt_tokenizer: HFTokenizerWrapper, max_len=100, device=DEVICE):
    """
    Translate a single sentence using the model with autoregressive generation.
    
    Args:
        sentence: Input sentence to translate
        model: TranslationTransformer model
        src_tokenizer: HFTokenizerWrapper for the source language
        tgt_tokenizer: HFTokenizerWrapper for the target language
        max_len: Maximum sequence length to generate
        device: Device to run on
        
    Returns:
        Translated sentence and token indices
    """
    model.eval()
    
    # Tokenize input
    src_tokens = src_tokenizer.tokenize(sentence)
    print(f"Input tokens: {src_tokens}")
    
    # Encode with EOS token only (source side)
    src_indices = src_tokenizer.encode(src_tokens, add_sos=False, add_eos=True)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)  # [1, seq_len]
    
    # Create source padding mask (all False since no padding in single sentence)
    src_key_padding_mask = torch.zeros(1, src_tensor.size(1), dtype=torch.bool).to(device)
    
    print(f"Input tensor shape: {src_tensor.shape}")
    print(f"Input indices: {src_indices}")
    
    # Initialize target with just SOS token
    tgt_indices = [tgt_tokenizer.sos_idx]
    
    # Autoregressive generation loop
    for _ in range(max_len):
        # Convert current target indices to tensor
        tgt_tensor = torch.tensor([tgt_indices], dtype=torch.long).to(device)  # [1, current_len]
        
        # Create target padding mask (all False since we're only generating, no padding)
        tgt_key_padding_mask = torch.zeros(1, tgt_tensor.size(1), dtype=torch.bool).to(device)
        
        # Forward pass with masks
        output = model(src_tensor, tgt_tensor, 
                      src_key_padding_mask=src_key_padding_mask,
                      tgt_key_padding_mask=tgt_key_padding_mask)  # [1, current_len, vocab_size]
        
        # Get prediction for the last token
        next_token_logits = output[0, -1, :]  # [vocab_size]
        next_token = torch.argmax(next_token_logits).item()
        
        # Append predicted token
        tgt_indices.append(next_token)
        
        # Stop if we predict EOS token
        if next_token == tgt_tokenizer.eos_idx:
            break
    
    print(f"Generated {len(tgt_indices)} tokens")
    print(f"Predicted indices: {tgt_indices}")
    
    # Decode back to tokens (skip SOS and EOS)
    translation = tgt_tokenizer.decode_to_text(tgt_indices)  # Remove SOS and EOS
        
    return translation, tgt_indices

# Test with a sample German sentence from the dataset
total_bleu = 0.0
print_enabled = False

batch_de, batch_en, _, _ = next(iter(test_loader))
samples = len(batch_de)

for idx, (de, en) in enumerate(zip(batch_de, batch_en)):
    sample_de = tokenizer.decode_to_text(de.tolist())
    sample_en = tokenizer.decode_to_text(en.tolist())

    translation, _ = translate_sample(
        sample_de, 
        model, 
        src_tokenizer=tokenizer,
        tgt_tokenizer=tokenizer,
        max_len=max_len,
        device=DEVICE
    )
    BLEUscore = sacrebleu.corpus_bleu([translation], [[sample_en]])
    total_bleu += BLEUscore.score

    if print_enabled:
        print(f"Original German: {sample_de}")
        print(f"Original English: {sample_en}")
        print("\n" + "="*60 + "\n")

        print("\n" + "="*60)
        print(f"\nModel Translation: '{translation}'")
        print(f"Reference Translation: {sample_en}")
        print(f"BLEU Score: {BLEUscore.score:.4f}")

print(f"\nAverage BLEU Score over {samples} samples: {(total_bleu / samples):.4f}")

In [ ]:

translation, _ = translate_sample(
    tokenizer.decode_to_text(test_ds[0][0].tolist()),
    model, 
    src_tokenizer=tokenizer,
    tgt_tokenizer=tokenizer,
    max_len=max_len,
    device=DEVICE
)
translation


## Saving and loading the model

To load the model later, use:
```python
checkpoint = torch.load('./models/translation_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
```